## Data Cleaning and Quality Control for Pregnancy Deep Phenotyping Metabolomics Data
##### Kayla Xu, Piekos Lab
##### 1/28/2026

In [1]:
# set up environment
import pandas as pd
import numpy as np
import scipy.stats as sp
from scipy import ndimage as nd
import logging
import sys

In [2]:
# read in csv files
exp = pd.read_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_expression.csv", index_col=0)
batch = pd.read_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_batch.csv", index_col=0)
comp = pd.read_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_compounds.csv", index_col=0)
#t_exp = pd.read_csv("/Users/kaylaxu/Desktop/data/clean_data/MTBL_placenta/pos_exp_test.csv")

1. Convert any blank, "NA", or 0 values for analyte measuremnts to standard missing value

In [3]:
def convert_missing(x):
    try:
        val = float(x)
        if val == 0:
            return np.nan
        else:
            return val
    except:
        return np.nan

exp = exp.map(convert_missing)
exp

,01,02,p1,p2,p3,p4,p5,p6,p7,p8,...,p3695,p3696,p3697,p3698,p3699,p3700,p3701,p3702,p3703,p3704
Sample_ID,,,,,,,,,,,,,,,,,,,,,
Pooled Cntrl,NaN,NaN,1.529698e+09,9.609349e+08,6.969368e+08,6.273636e+08,4.496688e+08,1.627703e+08,1.656823e+08,2.721866e+08,...,27488.284023,7203.652750,7677.504597,1342.107419,9610.930373,39079.646816,32186.048274,1601.922548,4823.045917,2013.510488
Pooled Cntrl,NaN,NaN,1.368645e+09,1.017308e+09,7.297922e+08,6.216437e+08,4.032618e+08,1.658927e+08,1.658927e+08,2.758879e+08,...,23529.385751,7018.815397,10839.720606,1334.575697,2107.552582,37595.243490,14915.134760,1483.575237,4975.115785,1822.346712
Pooled Cntrl,NaN,NaN,1.529095e+09,1.024928e+09,7.537896e+08,6.528907e+08,4.176116e+08,1.691492e+08,1.691136e+08,2.799403e+08,...,32523.477547,6763.429783,16598.414538,1365.399845,9541.553119,41865.668458,30473.225647,1565.337035,10850.491229,1234.040520
Pooled Cntrl,NaN,NaN,1.594295e+09,1.061045e+09,7.727009e+08,6.619380e+08,3.661752e+08,1.678279e+08,1.678279e+08,2.916086e+08,...,35486.556219,5281.358186,6002.694725,1960.021509,3823.302236,27640.514842,34969.834441,2063.441685,3151.114966,1291.215513
Pooled Cntrl,NaN,NaN,1.558062e+09,1.075289e+09,7.655375e+08,6.626260e+08,3.324415e+08,1.634159e+08,1.656492e+08,2.939343e+08,...,36362.884200,5176.081144,3629.736853,1781.069954,3374.391045,23745.862385,23107.849268,9204.205117,5361.216328,1412.393842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DP3-0409,399287.829141,1.818930e+07,1.354813e+09,1.521910e+08,5.321273e+08,1.617649e+07,4.847604e+08,2.317692e+06,2.798594e+06,6.827049e+07,...,2476.141617,1718.449783,47208.704015,21672.795377,13064.466449,30018.671239,3779.421277,112197.881939,2141.515525,4221.543097
DP3-0416,435095.699504,2.299017e+07,1.246003e+09,1.441815e+08,3.709081e+08,1.821745e+07,3.968532e+08,2.166746e+06,2.166746e+06,1.160527e+08,...,6645.010324,5135.848653,43469.674374,2321.027123,36859.753509,23856.126696,5112.783090,1738.518821,22113.724282,1038.665688
DP3-0419,272278.020734,1.864842e+07,1.280463e+09,2.599678e+08,7.597815e+08,2.831243e+07,6.925312e+08,2.621542e+06,2.621542e+06,2.159653e+08,...,12111.085549,13529.689066,21175.352211,12954.678928,51933.051727,50097.705991,5257.716925,7674.302380,8809.518392,1021.711583


2. Identify QC vs biological samples and separate these further by batch

In [4]:
is_pooled = ["Pooled" in s for s in exp.index]
pooled = exp.iloc[is_pooled,:]

is_sample = ["Pooled" not in s for s in exp.index]
sample_exp = exp.iloc[is_sample,:]


In [5]:
pooled['batch'] = batch["batch"][is_pooled]

In [6]:
sample_exp['batch'] = batch["batch"][is_sample]

In [7]:
unique_batches = batch["batch"].unique()
exp_data = {} # dictionary of metabolomic expression split by QC/biological sample and batch (e.g. id = "Pooled_32425")
for b in unique_batches:
    exp_data["Pooled_" + str(b)] = pooled[pooled['batch'] == b]
    exp_data["Samples_" + str(b)] = sample_exp[sample_exp['batch'] == b]

3. Check for the internal quality control of samples by using the standard run - these are the yellow boxes with red text in the files. For all biological samples, for each standard calculate the Median Absolute Deviation. FIlter out any sample that is >5 MAD (Threshold = Median - (5xMAD)). Track any samples that fail this test in a log file

In [8]:
# standard runs = 01 and 02, only in samples
batch1 = exp_data["Samples_32425"]
batch2 = exp_data["Samples_62323"]

b1_id = "32425"
b2_id = "62323"

In [9]:
# calculate MAD for each batch and standard run
mad_101 = sp.median_abs_deviation(batch1["01"])
mad_102 = sp.median_abs_deviation(batch1["02"])
mad_201 = sp.median_abs_deviation(batch2["01"])
mad_202 = sp.median_abs_deviation(batch2["02"])

# calculate median for each batch and standard run
med_101 = nd.median(batch1["01"])
med_102 = nd.median(batch1["02"])
med_201 = nd.median(batch2["01"])
med_202 = nd.median(batch2["02"])

In [10]:
# create mask of all values that don't pass filter
fail_101 = (batch1["01"] < med_101 - 5*mad_101) | (batch1["01"] > med_101 + 5*mad_101)
fail_102 = (batch1["02"] < med_102 - 5*mad_102) | (batch1["02"] > med_102 + 5*mad_102)

fail_201 = (batch2["01"] < med_201 - 5*mad_201) | (batch2["01"] > med_201 + 5*mad_201)
fail_202 = (batch2["02"] < med_202 - 5*mad_202) | (batch2["02"] > med_202 + 5*mad_202)


In [11]:
# generate log file
logging.basicConfig(
    filename='MTBL_cleaning.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'  # Use 'w' to overwrite the file each run, or 'a' to append
)
logging.info("Initializing metabolomics cleaning pipeline...")
#logging.warning("Missing metadata in row 45.")
#logging.error("Failed to load reference database.")


In [12]:
# remove samples that fail check
failed = batch1.index[fail_102]
for sample in failed:
    logging.warning("QC: Sample " + sample + "_" + b1_id + " failed Median Absolute Deviation threshold test.")
    batch1 = batch1.drop(index=sample)

4. Filter out any individual biological samples with >50% missingness. Track any samples that fail this test in a log file.

In [13]:
fail_missing = batch1.isna().sum(axis=1) > len(batch1.index)
f = batch1.index[fail_missing]
for s in f:
    logging.warning("QC: Sample " + s + " failed >50% missingness test.")
    batch1 = batch1.drop(index=s)

5. Calculate relative standard deviation (RSD) in QC Pools (RSD = SD/Mean * 100) - remove any metabolites with an RSD < 30%.

In [14]:
pool1 = exp_data["Pooled_32425"]
pool2 = exp_data["Pooled_62323"]

In [15]:
RSD1 = (pool1.std()/pool1.mean())*100 
RSD2 = (pool2.std()/pool2.mean())*100 

In [16]:
failed_RSD = RSD1.index[list(RSD1 > 30) or list(RSD2 > 30)]

In [ ]:
for m in failed_RSD:
    logging.warning("QC: Compound " + str(m) + " failed the RSD < 30 check\n" + 10*'\t' + "Pooled_32425 RSD = " + str(RSD1[m]) + "\n" + 10*'\t' + "Pooled_62323 RSD = " + str(RSD2[m]))
    pool1 = pool1.drop(columns=m)
    pool2 = pool2.drop(columns=m)
    batch1 = batch1.drop(columns=m)
    batch2 = batch2.drop(columns=m)

6. Remove any analyses with >20% missingness (note this can be done in the same looping step as the RSD QC check). Double-check that no group has signficantly differential patterns of missingness for discarded analyses